# Signed Graph Convolutional Network(SGCN)を用いたFraud User Detection

In [1]:
!git branch

  balance_theory
  balance_theory2
  master
  new_balance_theory2
* subgraph_training_theory2


```sh
pip install -r /home/ubuntu/SGCN/requirements.txt
pip install torch_scatter
pip install torch_sparse
pip install easydict
```


In [2]:
from sgcn import SignedGCNTrainer, SignedGCNPredictor
from parser import parameter_parser
from utils import tab_printer, read_graph, score_printer, save_logs
import easydict
import argparse
import pandas as pd
import numpy as np
import torch
import json
import networkx as nx
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import accuracy_score,confusion_matrix
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [ ]:
data_name = input('データセット：')

In [ ]:
args = easydict.EasyDict({
        "edge_path": f'../input/{data_name}/{data_name}_network.csv',#'../input/{data_name}/user_network.csv',
        "features_path":  f'../input/{data_name}/{data_name}_node_feature.csv',#'../input/{data_name}/user_network.csv',
        "nodes_path": f'../input/{data_name}/{data_name}_gt.csv',
        "embedding_path": f'../tmp/embedding/{data_name}_sgcn_feature05.pkl', # tmp folder for cross-validation
        "regression_weights_path": f'../tmp/weights/{data_name}_sgcn_feature05.pkl',
        "inductive_model_path": None, # f'../output/inductive/{data_name}_model', # or None
        "log_path": f'../logs/{data_name}_logs_feature05.json',
        "epochs":300,
        "test_size":0.33,
        "reduction_iterations": 128,
        "reduction_dimensions": 30,
        "seed": 42,
        "lamb": 0.0,
        "learning_rate": 0.001,  
        "weight_decay": 10e-4, 
        # "layers": [64, 32,16,8],
        "layers": [32, ],
        "spectral_features":False,
        "general_features": True,  
        "sample_num":None,
        "class_weights":False,
        "node_under_sampling":False,
        "hidden_residual":False,
        "eval_freq":1,
})

In [ ]:
display(pd.io.json.json_normalize(args).T)

## 10-fold cross-validation
- train : validation : test = 6:3:1

In [ ]:
#tab_printer(args)
edges, nodes_dict = read_graph(args) # nodes_dict['indice']:node_id , nodes_dict['label'] : label

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
kf = StratifiedKFold(n_splits=10)
all_indice = nodes_dict['indice']
all_labels = nodes_dict['label']
auc_scores = []

In [ ]:
for i, (train_index, test_index) in enumerate(kf.split(X=nodes_dict['indice'],y=nodes_dict['label'])):
    print("==== Training Phase ====")
    print(f'{i}-th fold')
    # training
    train_node_indice = all_indice[train_index]
    train_node_labels = all_labels[train_index]
    print(f'labels:{np.unique(train_node_labels,return_counts=True)}')
    tmp_nodes_dict = {}
    tmp_nodes_dict['all_ncount'] = nodes_dict['all_ncount']
    tmp_nodes_dict['indice'] = train_node_indice
    tmp_nodes_dict['label'] = train_node_labels
    trainer = SignedGCNTrainer(args, edges, tmp_nodes_dict)
    trainer.setup_dataset()
    trainer.create_and_train_model()
    
    if args.test_size > 0:
        # trainer.save_model() ## trainer.create_and_train_model()のなかで，すでにbest_modelが保存されている．
        # score_printer(trainer.logs)
        display(pd.DataFrame(trainer.logs['performance']))
        save_logs(args, trainer.logs)

    # test
    print("==== Test Phase ====")
    test_node_indice = all_indice[test_index]
    test_node_labels = all_labels[test_index]
    # feature = pd.read_csv(args.embedding_path,index_col='id').values
    feature = pd.read_pickle(args.embedding_path).drop('id',1).values
    test_feature = feature[test_node_indice]
    # weight = pd.read_csv(args.regression_weights_path)
    weight = pd.read_pickle(args.regression_weights_path)
    predictions = np.dot(test_feature,weight.values.T)
    probabilities = torch.nn.functional.softmax(torch.from_numpy(predictions)).numpy()
    predict_labels = probabilities.argmax(1)
    auc_score = roc_auc_score(y_true=[1 if i==-1 else 0 for i in test_node_labels],y_score=probabilities[:,1])
    auc_scores.append(auc_score)
    cmx = confusion_matrix(y_true=[1 if i==-1 else 0 for i in test_node_labels],y_pred=predict_labels)
    print(f"{i}-th fold's auc_score:{auc_score}")
    print(cmx)
    print()
    

In [ ]:
np.mean(auc_scores)

In [ ]:
_ = plt.hist(probabilities[:,1][test_node_labels==1],alpha=0.5,bins=10)
_ = plt.hist(probabilities[:,1][test_node_labels==-1],alpha=0.5,bins=10)
# plt.xlim(0,0.03)

# new_new_balance_theoryの結果
結果 (32,) (non-class-weights,10e-4, test_size=0.33,lamb=0,lr=0.001)　⬅️これに決定
- amazon :  0.8655989752374083
- alpha : 0.974480519480
- otc : 

結果 (32,16) (non-class-weights,10e-4, test_size=0.33,lamb=0,lr=0.001)　
- amazon :  0.876279480706
- alpha : 0.977922077922
- otc : 0.99072039072
- epinions : 0.9768200116


結果 (32,16, 8) (non-class-weights,10e-4, test_size=0.33, lamb=0)
- amazon : 0.8786657350643106
- alpha : 
- otc : 

結果 (32,16, 8, 8) (non-class-weights,10e-4, test_size=0.33, lamb=0)
- amazon : 0.8778590500661
- alpha : 
- otc : 



# new_balance_theoryの結果


結果 (32,16) (non-class-weights,10e-3, test_size=0.33)
- amazon : 0.8650228
- alpha : 0.984
- otc : 0.9979548

結果 (32,16, 8) (non-class-weights,10e-4, test_size=0.33)
- amazon : 0.8722951
- alpha : 
- otc : 

結果 (32,16) (non-class-weights,10e-3, test_size-0.5)
- amazon : 
- alpha : 
- otc : 

結果 (32,32,32,32) (non-class-weights,10e-3, hidden_residual,) (100 epoch)
- amazon : 0.86009158252
- alpha : 
- otc : 

結果 (32,32,32,) (class-weights,10e-3,hidden_residual) (100 epoch)
- amazon : 0.864321831
- alpha : 
- otc : 

---

結果 (32,) (no-class-weights, 0.33)
- amazon : 0.856184081
- alpha : 0.983
- otc : 0.998

結果 (32,16) (no-class-weights,test_size=0.33,weight_decay=10e-5)
- amazon : 0.871
- alpha : 0.9828
- otc : 0.997

結果 (32,16,8) (no-class-weights,test_size=0.33)
- amazon : 0.867
- alpha : 0.97123376
- otc : 0.9883699633699635

結果 (32,16,8) (no-class-weights,test_size=0.33,lamb=0)
- amazon : 0.871944
- alpha : 0.96454545
- otc : 0.994810744810

結果 (32,32,32) (no-class-weights,test_size=0.33,lamb=1.0,residual)
- amazon : 0.869
- alpha : 0.
- otc : 0.

結果 (32,16) (no-class-weights,test_size=0.33,weight_decay=10e-3)
- amazon : 0.871668935569  -->  0.866652196778459
- alpha : 0.9868181818181819 --> 0.9778071928071927
- otc : 0.9963980463980464  --> 0.99557387057

結果 (32,16) (no-class-weights,test_size=0.33,weight_decay=10e-3, amazon_unsigned)
- amazon : 
- alpha : 0.9868181818181819 --> 0.9778071928071927
- otc : 0.9963980463980464  --> 0.99557387057

---

結果 (32,16) (no-class-weights,test_size=0.66)
- amazon : 0.8635876758
- alpha : 0.9831
- otc : 0.993

結果 (64,32)
- amazon : 0.8597141032576031
- alpha : 
- otc : 

結果 (64,32,16,8)
- amazon : 0.8516649987979326
- alpha : 
- otc : 

結果
- amazon : 0.8704, (weighted classes loss : 0.871)
- alpha : (sampled: 0.9804), (normal: 0.9857)
- epinions : 
- otc : (0.9947), (normal:0.996)

## single-validation

In [ ]:
args = easydict.EasyDict({
        "edge_path": '../input/amazon/amazon_network.csv',#'../input/amazon/user_network.csv',
        "features_path":  '../input/amazon/amazon_node_feature.csv',#'../input/amazon/user_network.csv',
        "nodes_path": '../input/amazon/amazon_gt.csv',
        "embedding_path": '../tmp/embedding/amazon_sgcn_feature05.pkl', # tmp folder for cross-validation
        "regression_weights_path": '../tmp/weights/amazon_sgcn_feature05.pkl',
        "inductive_model_path": '../output/inductive/amazon_model', # or None
        "log_path": '../logs/amazon_logs_feature05.json',
        "epochs":500,
        "test_size":0.33,
        "reduction_iterations": 128,
        "reduction_dimensions": 30,
        "seed": 42,
        "lamb": 1.0,
        "learning_rate": 0.005,  
        "weight_decay": 10e-5, 
        # "layers": [64, 32,16,8],
        "layers": [32,16],
        "spectral_features":False,
        "general_features": True,  
        "sample_num":None,
        "class_weights":False,
        "node_under_sampling":False,
})

In [ ]:
#tab_printer(args)
edges, nodes_dict = read_graph(args) # nodes_dict['indice']:node_id , nodes_dict['label'] : label

In [ ]:
trainer = SignedGCNTrainer(args, edges, nodes_dict)
trainer.setup_dataset()
trainer.create_and_train_model()

In [ ]:
if args.test_size > 0:
    trainer.save_model()
    # score_printer(trainer.logs)
    display(pd.DataFrame(trainer.logs['performance']))
    save_logs(args, trainer.logs)

#### inductive settings

In [ ]:
new_args = easydict.EasyDict({
        "edge_path": '../input/amazon_music/amazon_music_network.csv',#'../input/amazon_music/user_network.csv',
        "features_path":  '../input/amazon_music/amazon_music_node_feature.csv',#'../input/amazon_music/user_network.csv',
        "nodes_path": '../input/amazon_music/amazon_music_gt.csv',
        "embedding_path": '../tmp/embedding/amazon_music_sgcn_feature05.csv', # tmp folder for cross-validation
        "regression_weights_path": '../tmp/weights/amazon_music_sgcn_feature05.csv',
        "inductive_model_path": None, # '../output/inductive/amazon_music_model', # or None
        "log_path": '../logs/amazon_music_logs_feature05.json',
        "epochs":300,
        "test_size":0.33,
        "reduction_iterations": 128,
        "reduction_dimensions": 30,
        "seed": 42,
        "lamb": 1.0,
        "learning_rate": 0.005,  
        "weight_decay": 10e-5, 
        # "layers": [64, 32,16,8],
        "layers": [32, 16],
        "spectral_features":False,
        "general_features": True,  
        "class_weights":False,
        "sample_num":None,
        "node_under_sampling":False,
})

In [ ]:
new_edges, new_nodes_dict = read_graph(new_args)

X = np.array(pd.read_csv('../input/amazon_elec/amazon_elec_node_feature.csv')) # general node features

In [ ]:
predictor = SignedGCNPredictor(new_args, '../output/inductive/amazon_model', X, new_edges,new_nodes_dict)

In [ ]:
predictions = predictor.predict()
predict_labels = predictions.argmax(1)

In [ ]:
y_true = new_nodes_dict['label']

In [ ]:
roc_auc_score(y_true=[1 if i==-1 else 0 for i in new_nodes_dict['label']],y_score=predictions[:,1][new_nodes_dict['indice']])

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
confusion_matrix([1 if i==-1 else 0 for i in new_nodes_dict['label']],predict_labels[new_nodes_dict['indice']])

In [ ]:
y_true.shape

In [ ]:
_ = plt.hist(predictions[:,1][new_nodes_dict['indice']][y_true==1],alpha=0.5,bins=10)
_ = plt.hist(predictions[:,1][new_nodes_dict['indice']][y_true==-1],alpha=0.5,bins=10)

結果
- amazon
    - elec : 0.80787063118
    - movie : 0.55
    - music : 0.76
    

In [ ]:
import json

In [ ]:
performance = pd.DataFrame(json.load(open('../logs/otc_logs_feature05.json','r'))['performance'])

performance.columns = performance.iloc[0,:]

performance = performance.iloc[1:,:]

In [ ]:
performance['AUC'].plot()

In [ ]:
feature = pd.read_csv(args.embedding_path,index_col='id')

weight = pd.read_csv(args.regression_weights_path)

In [ ]:
predictions = np.dot(feature.values,weight.values.T)

In [ ]:
probabilities = torch.nn.functional.softmax(torch.from_numpy(predictions)).numpy()

In [ ]:
predict_labels = probabilities.argmax(1)

In [ ]:
roc_auc_score(y_true=[0 if i==-1 else 1 for i in nodes_dict['label']],y_score=probabilities[:,1][nodes_dict['indice']])

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
confusion_matrix([0 if i==-1 else 1 for i in nodes_dict['label']],predict_labels[nodes_dict['indice']])

# TODO

#### for epinions dataset

In [3]:
data_name = 'epinions'

In [4]:
args = easydict.EasyDict({
        "edge_path": f'../input/{data_name}/{data_name}_network.csv',#'../input/{data_name}/user_network.csv',
        "features_path":  f'../input/{data_name}/{data_name}_node_feature.csv',#'../input/{data_name}/user_network.csv',
        "nodes_path": f'../input/{data_name}/{data_name}_gt.csv',
        "embedding_path": f'../tmp/embedding/{data_name}_sgcn_feature05.pkl', # tmp folder for cross-validation
        "regression_weights_path": f'../tmp/weights/{data_name}_sgcn_feature05.pkl',
        "inductive_model_path": f'../output/inductive/{data_name}_model', # or None
        "log_path": f'../logs/{data_name}_logs_feature05.json',
        "epochs":300,
        "test_size":0.33,
        "reduction_iterations": 128,
        "reduction_dimensions": 30,
        "seed": 42,
        "lamb": 0.0,
        "learning_rate": 0.001,  
        "weight_decay": 10e-4, 
        # "layers": [64, 32,16,8],
        "layers": [32, 16],
        "spectral_features":False,
        "general_features": True,  
        "sample_num":None,
        "class_weights":False,
        "node_under_sampling":False,
        "hidden_residual":False,
        "eval_freq":1,
        "subgraph_training":True,
})

In [5]:
display(pd.io.json.json_normalize(args).T)

,0
class_weights,False
edge_path,../input/epinions/epinions_network.csv
embedding_path,../tmp/embedding/epinions_sgcn_feature05.pkl
epochs,300
eval_freq,1
features_path,../input/epinions/epinions_node_feature.csv
general_features,True
hidden_residual,False
inductive_model_path,../output/inductive/epinions_model
lamb,0


In [6]:
#tab_printer(args)
edges, nodes_dict = read_graph(args) # nodes_dict['indice']:node_id , nodes_dict['label'] : label

In [7]:
test_X = np.array(pd.read_csv(args.features_path))

In [8]:
def subgraph_making(args,edges,nodes_dict,sample_node_num,train_node_indice_original):
    original_network_df = pd.read_csv(args.edge_path)

    sampled_node_indice = set(np.random.choice(np.arange(nodes_dict['all_ncount']), 20000, replace=False)) | set(train_node_indice_original)

    sub_network_df = \
        original_network_df.loc[(original_network_df.id1_.isin(sampled_node_indice)) & (original_network_df.id2_.isin(sampled_node_indice))]

    original_gt_df = pd.read_csv(args.nodes_path)

    sub_gt_df = \
        original_gt_df.copy().loc[(original_gt_df.node_id.isin(sub_network_df.id1_)) & (original_gt_df.node_id.isin(sub_network_df.id2_))]

    from sklearn.preprocessing import LabelEncoder

    subGraph_map_encoder = LabelEncoder()

    subGraph_map_encoder.fit(list(set(sub_network_df.id1_) | set(sub_network_df.id2_) | set(sub_gt_df.node_id)))

    sub_gt_df['node_id'] = subGraph_map_encoder.transform(sub_gt_df.node_id)

    sub_network_df['id1_'] = subGraph_map_encoder.transform(sub_network_df.id1_)

    sub_network_df['id2_'] = subGraph_map_encoder.transform(sub_network_df.id2_)

    sub_edges = {}

    sub_edges['positive_edges'] = sub_network_df.loc[sub_network_df.weight==1,['id1_','id2_']].values.tolist()

    sub_edges['negative_edges'] = sub_network_df.loc[sub_network_df.weight==-1,['id1_','id2_']].values.tolist()

    sub_edges['ecount'] = len(sub_network_df)

    sub_edges['ncount'] = len(set(sub_network_df['id1_']) | set(sub_network_df['id2_']))

    sub_nodes_dict = {}

    sub_nodes_dict['indice'] = sub_gt_df.node_id.values
    sub_nodes_dict['label'] = sub_gt_df.label.values

    sub_nodes_dict['all_ncount'] = len(set(sub_network_df.id1_) | set(sub_network_df.id2_) | set(sub_gt_df.node_id))

    sub_nodes_dict['subgraph_map_from_original_feature'] = subGraph_map_encoder.classes_    
    return sub_edges, sub_nodes_dict, subGraph_map_encoder

In [11]:
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=10)
all_indice = nodes_dict['indice']
all_labels = nodes_dict['label']
auc_scores = []

In [12]:
for i, (train_index, test_index) in enumerate(kf.split(X=nodes_dict['indice'],y=nodes_dict['label'])):
    print("==== Training Phase ====")
    print(f'{i}-th fold')
    # training
    train_node_indice_original = all_indice[train_index] #⬅️これは元々のグラフのnode_indice

    # extract sub-graph
    sub_edges, sub_nodes_dict,subGraph_map_encoder = subgraph_making(args,edges,nodes_dict,50000, [])
                                                                     # np.random.choice(train_node_indice_original,1000))
    
    train_indice_boolean_judger = np.isin(subGraph_map_encoder.inverse_transform(sub_nodes_dict['indice']),train_node_indice_original)
    train_node_indice = sub_nodes_dict['indice'][train_indice_boolean_judger]
    train_node_labels = sub_nodes_dict['label'][train_indice_boolean_judger]
    print(f'labels:{np.unique(train_node_labels,return_counts=True)}')
    tmp_nodes_dict = {}
    tmp_nodes_dict['all_ncount'] = sub_nodes_dict['all_ncount']
    tmp_nodes_dict['indice'] = train_node_indice
    tmp_nodes_dict['label'] = train_node_labels
    tmp_nodes_dict['subgraph_map_from_original_feature'] = subGraph_map_encoder.classes_
    trainer = SignedGCNTrainer(args, sub_edges, tmp_nodes_dict)
    trainer.setup_dataset()
    trainer.create_and_train_model()
    
    if args.test_size > 0:
        # trainer.save_model() ## trainer.create_and_train_model()のなかで，すでにbest_modelが保存されている．
        # score_printer(trainer.logs)
        display(pd.DataFrame(trainer.logs['performance']))
        save_logs(args, trainer.logs)

    # test
    print("==== Test Phase ====")
    test_node_indice = all_indice[test_index]
    test_node_labels = all_labels[test_index]
    predictor = SignedGCNPredictor(args, args.inductive_model_path, test_X, edges,nodes_dict)
    predictions = predictor.predict()
    predict_labels = predictions.argmax(1)
    auc_score = roc_auc_score(y_true=[1 if i==-1 else 0 for i in test_node_labels],y_score=predictions[:,1][test_node_indice])
    auc_scores.append(auc_score)
    cmx = confusion_matrix(y_true=[1 if i==-1 else 0 for i in test_node_labels],y_pred=predict_labels[test_node_indice])
    print(f"{i}-th fold's auc_score:{auc_score}")
    print(cmx)
    print()
    

==== Training Phase ====
0-th fold


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


labels:(array([-1,  1]), array([ 19, 728]))


SGCN (Loss=0.7162):   0%|          | 0/300 [00:00<?, ?it/s]


Training started.



...

SGCN (Loss=0.7138):   0%|          | 0/300 [00:00<?, ?it/s]


Training started.



SGCN (Loss=0.0248): 100%|██████████| 300/300 [00:59<00:00,  5.14it/s]


,0,1,2
0,Epoch,AUC,F1
1,1,0.554188,0
2,2,0.626042,0
3,3,0.688368,0
4,4,0.757443,0
5,5,0.802303,0
6,6,0.833664,0
7,7,0.850734,0
8,8,0.859865,0
9,9,0.870981,0


==== Test Phase ====
9-th fold's auc_score:0.9677399961523663
[[904   0]
 [ 18  74]]



In [13]:
np.mean(auc_scores)

0.9768200116159313